# Đọc dữ liệu từ HDFS

In [1]:
import json
from functools import reduce
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
            .master("spark://spark-master:7077") \
            .appName("Tutorial") \
            .getOrCreate()

In [2]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [ ]:
df = []
for i in range(10):
    df.append(spark.read.format("json")
              .load("hdfs://namenode/user/root/input/data{}.json".format(i+1), multiLine = "true"))

In [ ]:
df = unionAll(df[0], df[1], df[2], df[3], df[4], df[5], df[6], df[7], df[8], df[9])

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
df = df.na.drop().dropDuplicates()

In [ ]:
df = df.filter(size(df['comments']) >= 30)

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
df.groupby('androidVersion').count().sort('count', ascending=False).show()

In [ ]:
df = df.withColumn("androidVersion", col("androidVersion").cast("float"))

In [ ]:
df.groupby('androidVersion').count().sort('count', ascending=False).show()

In [ ]:
import re

def remove_character(size):
    """
        This UDF takes size as input and returns number of size
    """
    return re.sub(r'[^\d]', '', size)

convert_to_number = udf(remove_character, IntegerType())

In [ ]:
df = df.withColumn("size", convert_to_number(col("size")))
df.show(2)

In [ ]:
df.groupby('size').count().sort('count', ascending=False).show(10)

In [ ]:
spark.sql("Select size from dfTable")